In [1]:
import pandas as pd
import numpy as np
import random as random

# plotly standard imports
import plotly.graph_objs as go
import plotly.plotly as py



In [2]:
# Cufflinks wrapper on plotly
import cufflinks
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from plotly.offline import iplot
cufflinks.go_offline()

# Set global theme
cufflinks.set_config_file(world_readable=True, theme='pearl')
import plotly.figure_factory as ff


# Data input

In [256]:
input_df=pd.read_csv("train.csv", sep=",")
X=input_df.drop("revenue", axis=1) # drop labels for training set
y=input_df.revenue

In [257]:
input_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 23 columns):
id                       3000 non-null int64
belongs_to_collection    604 non-null object
budget                   3000 non-null int64
genres                   2993 non-null object
homepage                 946 non-null object
imdb_id                  3000 non-null object
original_language        3000 non-null object
original_title           3000 non-null object
overview                 2992 non-null object
popularity               3000 non-null float64
poster_path              2999 non-null object
production_companies     2844 non-null object
production_countries     2945 non-null object
release_date             3000 non-null object
runtime                  2998 non-null float64
spoken_languages         2980 non-null object
status                   3000 non-null object
tagline                  2403 non-null object
title                    3000 non-null object
Keywords             

In [258]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [259]:
X_train.reset_index(inplace=True);
X_test.reset_index(inplace=True);
y_train.reset_index;
y_train.reset_index;

In [260]:
df=pd.concat([X_train, y_train], axis=1, sort=False)

In [261]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2887 entries, 0 to 2998
Data columns (total 24 columns):
index                    2400 non-null float64
id                       2400 non-null float64
belongs_to_collection    484 non-null object
budget                   2400 non-null float64
genres                   2394 non-null object
homepage                 769 non-null object
imdb_id                  2400 non-null object
original_language        2400 non-null object
original_title           2400 non-null object
overview                 2392 non-null object
popularity               2400 non-null float64
poster_path              2399 non-null object
production_companies     2263 non-null object
production_countries     2350 non-null object
release_date             2400 non-null object
runtime                  2398 non-null float64
spoken_languages         2383 non-null object
status                   2400 non-null object
tagline                  1913 non-null object
title           

# Removing Nulls and Data exploration

In [262]:
X_train.isna().sum()

index                       0
id                          0
belongs_to_collection    1916
budget                      0
genres                      6
homepage                 1631
imdb_id                     0
original_language           0
original_title              0
overview                    8
popularity                  0
poster_path                 1
production_companies      137
production_countries       50
release_date                0
runtime                     2
spoken_languages           17
status                      0
tagline                   487
title                       0
Keywords                  217
cast                       11
crew                       14
dtype: int64

In [263]:
from sklearn.base import TransformerMixin

In [264]:
class DataFrameImputer(TransformerMixin):

    def __init__(self):
        """Impute missing values.

        Columns of dtype object are imputed with the most frequent value 
        in column.

        Columns of other types are imputed with mean of column.

        """
    def fit(self, X, y=None):

        self.fill = pd.Series([X[c].value_counts().index[0]
            if X[c].dtype == np.dtype('O') else X[c].mean() for c in X],
            index=X.columns)

        return self

    def transform(self, X, y=None):
        return X.fillna(self.fill)


In [265]:
X_train_t=DataFrameImputer().fit_transform(X_train)

In [266]:
df_t=DataFrameImputer().fit_transform(df)

In [267]:
X_train_t.isna().sum()

index                    0
id                       0
belongs_to_collection    0
budget                   0
genres                   0
homepage                 0
imdb_id                  0
original_language        0
original_title           0
overview                 0
popularity               0
poster_path              0
production_companies     0
production_countries     0
release_date             0
runtime                  0
spoken_languages         0
status                   0
tagline                  0
title                    0
Keywords                 0
cast                     0
crew                     0
dtype: int64

In [106]:
#Comparing correlatons between Consumption_per_NIA/log_NIA/Property Type 


figure = ff.create_scatterplotmatrix(
    df[['budget', 'popularity',"runtime","revenue"]],
    height=1000,
    width=1000,
    diag='histogram',
    title="Data Exploration")
iplot(figure)

In [268]:
corrs = df[["revenue","popularity","runtime","budget"]].corr()
figure = ff.create_annotated_heatmap(
    z=corrs.values,
    x=list(corrs.columns),
    y=list(corrs.index),
    annotation_text=corrs.round(2).values,
    showscale=True)

iplot(figure)

# Feature engineering

In [269]:
import ast


def cat_list(line):
    c_list=[]
    line_eval=ast.literal_eval(line)
    for d in line_eval:
        c_list.append(d["name"])
    return(c_list)

In [270]:
X_train_t.head()

,index,id,belongs_to_collection,budget,genres,homepage,imdb_id,original_language,original_title,overview,...,production_countries,release_date,runtime,spoken_languages,status,tagline,title,Keywords,cast,crew
0,642,643,"[{'id': 645, 'name': 'James Bond Collection', ...",39000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",http://www.transformersmovie.com/,tt0094898,en,Coming to America,"Prince Akeem, heir to the throne of Zamunda, l...",...,"[{'iso_3166_1': 'US', 'name': 'United States o...",6/28/88,116.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The Four Funniest Men in America are Eddie Mur...,Coming to America,"[{'id': 3015, 'name': 'fast food restaurant'},...","[{'cast_id': 1, 'character': 'Prince Akeem / C...","[{'credit_id': '52fe450fc3a36847f80ba1d3', 'de..."
1,700,701,"[{'id': 645, 'name': 'James Bond Collection', ...",25000000,"[{'id': 27, 'name': 'Horror'}, {'id': 35, 'nam...",http://www.transformersmovie.com/,tt0091419,en,Little Shop of Horrors,Seymour Krelborn is a nerdy orphan working at ...,...,"[{'iso_3166_1': 'US', 'name': 'United States o...",12/19/86,94.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Don't feed the plants.,Little Shop of Horrors,"[{'id': 2382, 'name': 'flower'}, {'id': 2642, ...","[{'cast_id': 8, 'character': 'Seymour Krelborn...","[{'credit_id': '52fe43b59251416c7501b029', 'de..."
2,226,227,"[{'id': 645, 'name': 'James Bond Collection', ...",50000000,"[{'id': 14, 'name': 'Fantasy'}, {'id': 53, 'na...",http://www.driveangry3d.com/,tt1502404,en,Drive Angry,Milton is a hardened felon who has broken out ...,...,"[{'iso_3166_1': 'US', 'name': 'United States o...",2/24/11,105.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,One hell of a ride.,Drive Angry,"[{'id': 4479, 'name': 'bone'}, {'id': 41019, '...","[{'cast_id': 4, 'character': 'Milton', 'credit...","[{'credit_id': '54e07d43c3a368454d0053e5', 'de..."
3,1697,1698,"[{'id': 645, 'name': 'James Bond Collection', ...",0,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",http://www.transformersmovie.com/,tt0105391,en,Shining Through,Spirited New Yorker Linda Voss goes to work fo...,...,"[{'iso_3166_1': 'GB', 'name': 'United Kingdom'...",1/31/92,132.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,He needed to trust her with his secret. She ha...,Shining Through,"[{'id': 74, 'name': 'germany'}, {'id': 818, 'n...","[{'cast_id': 2, 'character': 'Ed Leland', 'cre...","[{'credit_id': '58fd30ef925141643c02713f', 'de..."
4,1010,1011,"[{'id': 645, 'name': 'James Bond Collection', ...",35000000,"[{'id': 35, 'name': 'Comedy'}]",http://www.transformersmovie.com/,tt3152624,en,Trainwreck,Having thought that monogamy was never possibl...,...,"[{'iso_3166_1': 'US', 'name': 'United States o...",7/17/15,125.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,We All Know One.,Trainwreck,"[{'id': 567, 'name': 'alcohol'}, {'id': 2864, ...","[{'cast_id': 4, 'character': 'Amy Townsend', '...","[{'credit_id': '55aa7f0c92514172ef003b26', 'de..."


In [271]:
from sklearn.base import BaseEstimator, TransformerMixin

In [272]:
df.columns

Index(['index', 'id', 'belongs_to_collection', 'budget', 'genres', 'homepage',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'runtime', 'spoken_languages',
       'status', 'tagline', 'title', 'Keywords', 'cast', 'crew', 'revenue'],
      dtype='object')

# Unpacking categories and spoken languages

In [320]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()

class CatAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self):
        """Impute missing values.

        Columns of dtype object are imputed with the most frequent value 
        in column.

        Columns of other types are imputed with mean of column.

        """
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        genres_list=X["genres"].apply(lambda x: cat_list(x))
        genres_label_data=mlb.fit_transform(genres_list)
        genres_labelClasses = mlb.classes_
        genresLabels=pd.DataFrame(genres_label_data, columns=genres_labelClasses)
        
        language_list=X['spoken_languages'].apply(lambda x: cat_list(x))
        language_label_data=mlb.fit_transform(language_list)
        language_labelClasses = mlb.classes_
        languageLabels=pd.DataFrame(language_label_data, columns=language_labelClasses)
        languageLabels=languageLabels[[ 'Deutsch', 'English', 'Español', 'Français', 'Italiano', 'Pусский']]
        
        
        
        processed_X=pd.concat([X, genresLabels,languageLabels], axis=1, sort=False)
        
        return(processed_X)
        
        
     

In [321]:
attr_adder=CatAttributesAdder()
X_train_t2=attr_adder.fit_transform(X_train_t)

In [322]:
X_train_t2.columns

Index(['index', 'id', 'belongs_to_collection', 'budget', 'genres', 'homepage',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'runtime', 'spoken_languages',
       'status', 'tagline', 'title', 'Keywords', 'cast', 'crew', 'Action',
       'Adventure', 'Animation', 'Comedy', 'Crime', 'Documentary', 'Drama',
       'Family', 'Fantasy', 'Foreign', 'History', 'Horror', 'Music', 'Mystery',
       'Romance', 'Science Fiction', 'TV Movie', 'Thriller', 'War', 'Western',
       'Deutsch', 'English', 'Español', 'Français', 'Italiano', 'Pусский'],
      dtype='object')

In [342]:
len(X_train_t2)

2400

In [343]:
X_train_t2.isna().sum()

index                    0
id                       0
belongs_to_collection    0
budget                   0
genres                   0
homepage                 0
imdb_id                  0
original_language        0
original_title           0
overview                 0
popularity               0
poster_path              0
production_companies     0
production_countries     0
release_date             0
runtime                  0
spoken_languages         0
status                   0
tagline                  0
title                    0
Keywords                 0
cast                     0
crew                     0
Action                   0
Adventure                0
Animation                0
Comedy                   0
Crime                    0
Documentary              0
Drama                    0
Family                   0
Fantasy                  0
Foreign                  0
History                  0
Horror                   0
Music                    0
Mystery                  0
R

In [344]:
def get_left(string):
    try:
        if len(string)==8:
            return(int(string[:2]))
        else:
            return(int(string[:1]))
    except: return 0

In [345]:

def get_year(string):
    try:
        if int(string[-2:])< 20:
            return int(string[-2:])+2000
        else:
            return int(string[-2:])+1900
    except:
        return 0

# Adding new features

In [359]:
class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self):
        """

        init
        """
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
       
        year=X['release_date'].apply(lambda x: get_year(x))
        df_year=pd.DataFrame(year.values,columns=["year"])
        month=X['release_date'].apply(lambda x:get_left(x))
        df_month=pd.DataFrame(month.values,columns=["month"])
        X=X.drop(['index', 'id', 'belongs_to_collection', 'genres', 'homepage',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'poster_path', 'production_companies',
       'production_countries', 'release_date', 'spoken_languages',
       'status', 'tagline', 'title', 'Keywords', 'cast', 'crew'], axis=1)
        processed_X=pd.concat([X, df_year, df_month], axis=1, sort=False)
        processed_X=processed_X[['budget', 'popularity', 'runtime', 'Action', 'Adventure', 'Animation',
       'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy',
       'Foreign', 'History', 'Horror', 'Music', 'Mystery', 'Romance',
       'Science Fiction', 'Thriller', 'War', 'Western', 'Deutsch', 'English',
       'Español', 'Français', 'Italiano', 'Pусский', 'year', 'month']]

        return processed_X
  

In [347]:
attr_adder=CombinedAttributesAdder()
X_train_t3=attr_adder.transform(X_train_t2)


In [348]:
len(X_train_t3)

2400

In [349]:
X_train_t3.columns

Index(['budget', 'popularity', 'runtime', 'Action', 'Adventure', 'Animation',
       'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy',
       'Foreign', 'History', 'Horror', 'Music', 'Mystery', 'Romance',
       'Science Fiction', 'TV Movie', 'Thriller', 'War', 'Western', 'Deutsch',
       'English', 'Español', 'Français', 'Italiano', 'Pусский', 'year',
       'month'],
      dtype='object')

In [350]:
X_train_t3.isna().sum()

budget             0
popularity         0
runtime            0
Action             0
Adventure          0
Animation          0
Comedy             0
Crime              0
Documentary        0
Drama              0
Family             0
Fantasy            0
Foreign            0
History            0
Horror             0
Music              0
Mystery            0
Romance            0
Science Fiction    0
TV Movie           0
Thriller           0
War                0
Western            0
Deutsch            0
English            0
Español            0
Français           0
Italiano           0
Pусский            0
year               0
month              0
dtype: int64

# Preparation Pipeline

In [360]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

preparation_pipeline = Pipeline([
    ("imputer", DataFrameImputer()),
    ("cat_adder", CatAttributesAdder()),
    ("attribs_adder", CombinedAttributesAdder()), 
    ])

In [361]:
X_train_prep=preparation_pipeline.fit_transform(X_train)

In [362]:
X_train_prep.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2400 entries, 0 to 2399
Data columns (total 30 columns):
budget             2400 non-null int64
popularity         2400 non-null float64
runtime            2400 non-null float64
Action             2400 non-null int32
Adventure          2400 non-null int32
Animation          2400 non-null int32
Comedy             2400 non-null int32
Crime              2400 non-null int32
Documentary        2400 non-null int32
Drama              2400 non-null int32
Family             2400 non-null int32
Fantasy            2400 non-null int32
Foreign            2400 non-null int32
History            2400 non-null int32
Horror             2400 non-null int32
Music              2400 non-null int32
Mystery            2400 non-null int32
Romance            2400 non-null int32
Science Fiction    2400 non-null int32
Thriller           2400 non-null int32
War                2400 non-null int32
Western            2400 non-null int32
Deutsch            2400 non-nul

In [363]:
X_train_prep.head()

,budget,popularity,runtime,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,...,War,Western,Deutsch,English,Español,Français,Italiano,Pусский,year,month
0,39000000,7.830087,116.0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,1988,6
1,25000000,7.409179,94.0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,1986,12
2,50000000,7.668829,105.0,1,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,2011,2
3,0,5.860319,132.0,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,1992,1
4,35000000,17.310647,125.0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,2015,7


In [364]:
len(X_train_prep)

2400

In [365]:
X_train_prep.isna().sum()

budget             0
popularity         0
runtime            0
Action             0
Adventure          0
Animation          0
Comedy             0
Crime              0
Documentary        0
Drama              0
Family             0
Fantasy            0
Foreign            0
History            0
Horror             0
Music              0
Mystery            0
Romance            0
Science Fiction    0
Thriller           0
War                0
Western            0
Deutsch            0
English            0
Español            0
Français           0
Italiano           0
Pусский            0
year               0
month              0
dtype: int64

In [366]:
X_test_prep=preparation_pipeline.fit_transform(X_test)

In [367]:
X_test_prep.columns

Index(['budget', 'popularity', 'runtime', 'Action', 'Adventure', 'Animation',
       'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy',
       'Foreign', 'History', 'Horror', 'Music', 'Mystery', 'Romance',
       'Science Fiction', 'Thriller', 'War', 'Western', 'Deutsch', 'English',
       'Español', 'Français', 'Italiano', 'Pусский', 'year', 'month'],
      dtype='object')

In [368]:
len(X_test_prep)

600

In [369]:
X_test_prep.isna().sum()

budget             0
popularity         0
runtime            0
Action             0
Adventure          0
Animation          0
Comedy             0
Crime              0
Documentary        0
Drama              0
Family             0
Fantasy            0
Foreign            0
History            0
Horror             0
Music              0
Mystery            0
Romance            0
Science Fiction    0
Thriller           0
War                0
Western            0
Deutsch            0
English            0
Español            0
Français           0
Italiano           0
Pусский            0
year               0
month              0
dtype: int64

# Linear regression

In [370]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(X_train_prep, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [374]:
from sklearn.metrics import mean_squared_error

reve_pred=lin_reg.predict(X_test_prep)
lin_mse = mean_squared_error(reve_pred, y_test)
lin_rmse=np.sqrt(lin_mse)
lin_rmse

79949057.97195558

In [375]:
lin_rmse/y_test.mean()

1.1253323360648442

In [385]:
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor(n_estimators=10, random_state=42)
forest_reg.fit(X_train_prep, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=42, verbose=0, warm_start=False)

In [387]:
reve_pred=forest_reg.predict(X_test_prep)
rfr_mse = mean_squared_error(reve_pred, y_test)
rfr_rmse=np.sqrt(rfr_mse)
rfr_rmse

71384670.54371251

In [388]:
rfr_rmse/y_test.mean()

1.0047832970133788

In [389]:
from sklearn.model_selection import GridSearchCV

In [390]:
param_grid = [
    {'n_estimators':[3,10,30], "max_features":[2,4,6,8]},
    
    {"bootstrap":[False], "n_estimators":[3,10],"max_features":[2,3,4]},
    
]

In [391]:
forest_reg=RandomForestRegressor (random_state=10)

grid_search = GridSearchCV(forest_reg, param_grid, cv=5,
                          scoring='neg_mean_squared_error',return_train_score=True)

grid_search.fit(X_train_prep, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
           oob_score=False, random_state=10, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid=[{'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]}, {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='neg_mean_squared_error', verbose=0)

In [392]:
grid_search.best_params_

{'max_features': 8, 'n_estimators': 30}

In [393]:
feature_importances = grid_search.best_estimator_.feature_importances_
feature_importances

array([3.97980169e-01, 2.44506346e-01, 7.90074346e-02, 1.12987881e-02,
       3.79783397e-02, 6.98020687e-03, 1.07632419e-02, 5.32326961e-03,
       1.35923846e-04, 1.41500240e-02, 7.30712750e-03, 1.00120819e-02,
       3.46794904e-05, 2.55281368e-03, 4.89648205e-03, 2.56421735e-03,
       4.07340665e-03, 7.27693090e-03, 1.68633241e-02, 6.17890895e-03,
       1.24310837e-03, 1.28215090e-03, 4.17216104e-03, 5.91570636e-03,
       6.17908226e-03, 4.91476164e-03, 1.74156401e-03, 2.36914826e-03,
       5.69534989e-02, 4.53451019e-02])

In [397]:
attributes = X_train_prep.columns

In [398]:
sorted(zip(feature_importances, attributes), reverse=True)

[(0.3979801690929402, 'budget'),
 (0.24450634610992095, 'popularity'),
 (0.07900743456440538, 'runtime'),
 (0.05695349887140567, 'year'),
 (0.04534510187633235, 'month'),
 (0.03797833971563689, 'Adventure'),
 (0.016863324100485917, 'Science Fiction'),
 (0.014150023996893515, 'Drama'),
 (0.01129878810890743, 'Action'),
 (0.01076324190981531, 'Comedy'),
 (0.010012081928777736, 'Fantasy'),
 (0.007307127500175221, 'Family'),
 (0.007276930898034968, 'Romance'),
 (0.006980206866133815, 'Animation'),
 (0.006179082261737725, 'Español'),
 (0.0061789089496983455, 'Thriller'),
 (0.005915706359665607, 'English'),
 (0.005323269609021293, 'Crime'),
 (0.004914761636845926, 'Français'),
 (0.004896482052946362, 'Horror'),
 (0.004172161043473512, 'Deutsch'),
 (0.004073406647878031, 'Mystery'),
 (0.002564217348589863, 'Music'),
 (0.0025528136753804535, 'History'),
 (0.0023691482620688833, 'Pусский'),
 (0.0017415640142844128, 'Italiano'),
 (0.0012821508964283893, 'Western'),
 (0.0012431083656458547, 'War'

In [399]:

final_model = grid_search.best_estimator_

In [400]:
final_pred=final_model.predict(X_test_prep)

In [402]:
final_mse = mean_squared_error(y_test, final_pred)
final_rmse=np.sqrt(final_mse_enchanced)

In [403]:
final_rmse

69546536.91296713

In [404]:
final_rmse/y_test.mean()

0.9789104316518926